In [43]:
import tensorflow as tf

class CNNModel:
    def __init__(self, input_dim):
        self.layers = []
        self.input_dim = input_dim
        self.prev_dim = input_dim  # (H, W, C)

    def add_dense(self, units, activation='relu'):
        flat_dim = tf.reduce_prod(self.prev_dim)
        
        W = tf.Variable(tf.random.normal([flat_dim, units], stddev=0.1), trainable=True)
        b = tf.Variable(tf.zeros([units]), trainable=True)
        
        self.layers.append({'W': W, 'b': b, 'activation': activation, 'type': 'dense'})
        self.prev_dim = (units,)

    def add_convolution(self, filters, kernel_size, stride=1, padding=0, activation='relu'):
        height, width, in_channels = self.prev_dim
        k_h, k_w = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        
        K = tf.Variable(tf.random.normal([filters, k_h, k_w, in_channels], stddev=0.1), trainable=True)
        b = tf.Variable(tf.zeros([filters]), trainable=True)

        self.layers.append({
            'W': K, 'b': b, 'stride': stride, 'padding': padding,
            'activation': activation, 'type': 'conv'
        })

        h_out = (height + 2 * padding - k_h) // stride + 1
        w_out = (width + 2 * padding - k_w) // stride + 1
        self.prev_dim = (h_out, w_out, filters)

    def add_pooling(self, pool_size=2, stride=2):
        self.layers.append({'pool_size': pool_size, 'stride': stride, 'type': 'pool'})
        
        h, w, c = self.prev_dim
        
        h_out = (h - pool_size) // stride + 1
        w_out = (w - pool_size) // stride + 1
        self.prev_dim = (h_out, w_out, c)

    def _convolve(self, X, K, stride, padding):
        batch_size, height, width, in_channels = X.shape
        out_channels, k_h, k_w, _ = K.shape

        if padding > 0:
            X = tf.pad(X, [[0, 0], [padding, padding], [padding, padding], [0, 0]])

        h_out = (height + 2 * padding - k_h) // stride + 1
        w_out = (width + 2 * padding - k_w) // stride + 1
        
        output = tf.TensorArray(dtype=tf.float32, size=out_channels)
        for oc in range(out_channels):
            kernel = K[oc]
            out = []
            for b in range(batch_size):
                single_out = []
                for i in range(h_out):
                    row = []
                    for j in range(w_out):
                        region = X[b, i*stride:i*stride+k_h, j*stride:j*stride+k_w, :]
                        val = tf.reduce_sum(region * kernel)
                        row.append(val)
                    single_out.append(row)
                out.append(single_out)
            output = output.write(oc, tf.convert_to_tensor(out))

        output = tf.transpose(output.stack(), perm=[1, 2, 3, 0])
        return output

    def _max_pool(self, X, pool_size, stride):
        batch_size, height, width, channels = X.shape
        
        h_out = (height - pool_size) // stride + 1
        w_out = (width - pool_size) // stride + 1

        pooled = []
        for b in range(batch_size):
            img = X[b]
            pooled_img = []
            for i in range(h_out):
                row = []
                for j in range(w_out):
                    region = img[i*stride:i*stride+pool_size, j*stride:j*stride+pool_size, :]
                    row.append(tf.reduce_max(region, axis=[0, 1]))
                pooled_img.append(row)
            pooled.append(pooled_img)

        return tf.convert_to_tensor(pooled)

    def forward(self, X):
        out = X
        for layer in self.layers:
            if layer['type'] == 'conv':
                Z = self._convolve(out, layer['W'], layer['stride'], layer['padding'])
                Z = tf.nn.bias_add(Z, layer['b'])
                out = self._apply_activation(Z, layer['activation'])
            elif layer['type'] == 'pool':
                out = self._max_pool(out, layer['pool_size'], layer['stride'])
            elif layer['type'] == 'dense':
                out = tf.reshape(out, [out.shape[0], -1])
                Z = tf.matmul(out, layer['W']) + layer['b']
                out = self._apply_activation(Z, layer['activation'])

        return out

    def _apply_activation(self, Z, activation):
        if activation == 'relu':
            return tf.nn.relu(Z)
        elif activation == 'sigmoid':
            return tf.nn.sigmoid(Z)
        elif activation == 'tanh':
            return tf.nn.tanh(Z)
        else:
            return Z

    def train(self, X, Y, epochs=100, lr=0.01, loss_fn='mse'):
        optimizer = tf.optimizers.Adam(lr)

        for epoch in range(epochs):
            with tf.GradientTape() as tape:
                predictions = self.forward(X)
                if loss_fn == 'mse':
                    loss = tf.reduce_mean((predictions - Y) ** 2)
                elif loss_fn == 'bce':
                    loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(Y, predictions))
                else:
                    raise ValueError("Unsupported loss")

            variables = []
            for layer in self.layers:
                if 'W' in layer:
                    variables.append(layer['W'])
                if 'b' in layer:
                    variables.append(layer['b'])

            grads = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(grads, variables))

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")

    def predict(self, X):
        return self.forward(X)

In [45]:
model = CNNModel(input_dim=(8, 8, 1))  
model.add_convolution(filters=2, kernel_size=3, stride=1, padding=0, activation='relu')
model.add_pooling(pool_size=2, stride=2)
model.add_dense(units=10, activation='relu')
model.add_dense(units=1, activation='sigmoid')

X = tf.random.normal([5, 8, 8, 1])
Y = tf.constant([[1.], [0.], [1.], [0.], [1.]])

model.train(X, Y, epochs=100, lr=0.01, loss_fn='bce')

Epoch 0, Loss: 0.6911
Epoch 10, Loss: 0.6266
Epoch 20, Loss: 0.4516
Epoch 30, Loss: 0.1758
Epoch 40, Loss: 0.0277
Epoch 50, Loss: 0.0042
Epoch 60, Loss: 0.0011
Epoch 70, Loss: 0.0004
Epoch 80, Loss: 0.0002
Epoch 90, Loss: 0.0001
